In [2]:
pip install pygame

In [4]:
pip install opencv-python


   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB 1.4 MB/s eta 0:00:29
   ---------------------------------------- 0.3/38.6 MB 5.7 MB/s eta 0:00:07
   - -------------------------------------- 1.7/38.6 MB 18.5 MB/s eta 0:00:02
   --- ------------------------------------ 3.6/38.6 MB 28.9 MB/s eta 0:00:02
   ----- ---------------------------------- 5.5/38.6 MB 35.3 MB/s eta 0:00:01
   ------- -------------------------------- 7.4/38.6 MB 39.7 MB/s eta 0:00:01
   --------- ------------------------------ 9.2/38.6 MB 39.2 MB/s eta 0:00:01
   ----------- ---------------------------- 10.9/38.6 MB 50.4 MB/s eta 0:00:01
   ------------- -------------------------- 12.9/38.6 MB 54.4 MB/s eta 0:00:01
   --------------- ------------------------ 14.8/38.6 MB 50.4 MB/s eta 0:00:01
   ----------------- ---------------------- 16.7/38.6 MB 50.4 MB/s eta 0:00:01
   ------------------- -------------------- 18.5/38.6 MB 54.4 MB/s et

In [18]:
import pygame
import cv2
import numpy as np
import ctypes

# Set webcam window position and size
def set_window_position(window_name, x, y, width, height):
    win32gui = ctypes.windll.user32
    hwnd = win32gui.FindWindowW(None, window_name)
    if hwnd != 0:
        win32gui.MoveWindow(hwnd, x, y, width, height, True)

# Pygame setup
pygame.init()
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
size = [800, 600]  # Adjusted screen size for better display
screen = pygame.display.set_mode(size)
done = False
clock = pygame.time.Clock()
score = 0

# Function to run the game
def runGame():
    global done, score
    screen_width = size[0]
    screen_height = size[1]
    bar_width = 9
    bar_height = 100  # Adjusted for better visibility
    bar_x = bar_start_x = 0
    bar_y = bar_start_y = (screen_height - bar_height) / 2
    circle_radius = 9
    circle_diameter = circle_radius * 2
    circle_x = circle_start_x = screen_width - circle_diameter
    circle_y = circle_start_y = (screen_height - circle_diameter) / 2
    speed_x, speed_y, speed_bar = -screen_width / 2.56, screen_height / 3.84, screen_height * 0.6  # Adjusted ball speed

    # OpenCV setup for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)

    # Create a separate window for webcam feed
    cv2.namedWindow("Webcam", cv2.WINDOW_NORMAL)
    set_window_position("Webcam", 850, 100, 640, 480)

    while not done:
        time_passed = clock.tick(30)
        time_sec = time_passed / 1000.0
        screen.fill(BLACK)

        circle_x += speed_x * time_sec
        circle_y += speed_y * time_sec
        ai_speed = speed_bar * time_sec

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True
                break

        # Read frame from webcam
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        # Process detected faces
        for (x, y, w, h) in faces:
            # Calculate face position relative to screen center
            face_center_x = x + w // 2
            face_center_y = y + h // 2
            screen_center_y = size[1] // 2

            # Adjust paddle position based on face position
            bar_y = (screen_height - bar_height) * (face_center_y / 480)

            # Limit paddle movement to webcam frame boundaries
            if bar_y < 0:
                bar_y = 0
            elif bar_y > screen_height - bar_height:
                bar_y = screen_height - bar_height

            # Adjust paddle direction based on face position
            if face_center_y < screen_center_y:
                speed_bar = -abs(speed_bar)
            else:
                speed_bar = abs(speed_bar)

        # Draw webcam frame on separate window
        cv2.imshow("Webcam", frame)

        # Check collision between ball and paddle
        if circle_x < bar_x + bar_width and circle_x + circle_diameter > bar_x:
            if circle_y < bar_y + bar_height and circle_y + circle_diameter > bar_y:
                speed_x = -speed_x
                score += 1

        if circle_x < 0:
            circle_x, circle_y = circle_start_x, circle_start_y
            bar_x, bar_y = bar_start_x, bar_start_y
            score = 0
        elif circle_x > screen_width - circle_diameter:
            speed_x = -speed_x
        if circle_y <= 0:
            speed_y = -speed_y
            circle_y = 0
        elif circle_y >= screen_height - circle_diameter:
            speed_y = -speed_y
            circle_y = screen_height - circle_diameter

        # Draw score on the screen
        font = pygame.font.Font(None, 36)
        text = font.render(f"Score: {score}", True, WHITE)
        screen.blit(text, (10, 10))

        pygame.draw.rect(screen, WHITE, (bar_x, bar_y, int(bar_width), int(bar_height)))
        pygame.draw.circle(screen, WHITE, (int(circle_x), int(circle_y)), int(circle_radius))

        pygame.display.update()

    cap.release()
    cv2.destroyAllWindows()

runGame()
pygame.quit()
